## 1. 概述

许多事件的发展都依赖于连续的决策，如棋局中的每步选择或问卷调查中基于前述回答的后续问题选择。在有监督机器学习中，很多场景里的预测问题也遵循一系列决策条件。

决策树是这种连续决策模型的代表。它利用一系列决策条件将训练数据集划分为互不相交的子集，并为每个子集拟合一个函数。这些决策条件在树状结构中呈现，其中每个条件和子集分别对应决策树的中间结点和叶子结点。通常，每个叶子上的数据都被常数函数拟合。

决策树有悠久的历史。1963年，Morgan和Sonquist首先提议使用决策树解决回归问题，并设计了自顶向下的训练方法。1972年，这一思想被Messenger和Mandell应用于分类问题。1984年，CART算法的提出大大推进了决策树的发展，它利用验证集后剪枝，有效地控制树的规模并增强测试集的泛化能力。此后，决策树得到了许多扩展和变种，包括使用线性模型或多项式模型拟合叶子结点，或采用不同的回归目标函数。

决策树的结构明了，具备强大的可解释性，与需大量计算的深度神经网络形成鲜明对比。尽管深度学习的兴起使决策树在大数据环境下显得有些局限，但其基于的集成学习算法，如梯度提升树和随机森林，仍在表格数据学习中占有重要地位。高性能的工具包，如XGBoost和LightGBM，也确保了决策树在机器学习中的地位。同时，一些深度模型如深度随机森林和多层梯度提升树也为机器学习领域提供了新的探索方向。因此，决策树仍是一个值得深入研究的领域。


## 2. 决策树基本模型结构

给定有监督学习数据$D=(X,Y)=\{(x_i,y_i ) |  1 \leq i \leq n \}$,其中$x_i = \{x_{ij} |  1 \leq j \leq m \}$，即每个样本都由 $m$ 个特征 $x_{i1}, ..., x_{im}$ 组成。针对不同的任务标签 $y_i$ 会有所不同，例如对于回归任务 $y_i$ 是实数，对于分类任务 $y_i$ 是离散值。

图1展示了决策树的一个示例。决策树由中间结点和叶子结点组成。中间结点又称为决策结点，每个结点表示一个判断条件，其引出的每个分支代表一种判断结果，指向不同的子结点。因此，决策树通过中间结点的判断条件，将数据所在的空间切分成对应叶子结点的多个子空间。一般情况下，我们只让每个中间结点引出两个分支。这样可以把决策树都简化成二叉树，更方便训练及使用。并且，多叉树本质上也可以通过二叉树表达。在没有特殊说明的情况下，本书里的决策树都表示二叉决策树。

---

<center> <img src=https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/439/86060094db854f26a8af178070c33ae6/U3Qjr8b4Xs0466Q6y79U5g.png> </center>

<center> 图1 决策树模型 </center>

---

叶子结点又称为终结结点，每个结点会有个输出函数，用于赋予预测样本的输出值。给定一个样本$x$，决策树从根结点（顶部的中间结点）出发，依次经过中间结点的判断，会到达一个叶子结点。然后，由该叶子结点上的输出函数给出对于样本$x$的预测。因此，给定一个数据集，决策树会把这个数据集，按照中间结点的决策规则，把样本切分到决策树上的各个叶子上。
中间结点的判断条件既可以用数值特征，也可以用类别特征。对于数值特征值$x_{ij}$，将其值与一个阈值t进行比较，如果$x_{ij}\leq t$则$x_i$被分到左子结点，否则分到右子结点。对于类别特征$x_{ij}$，假设其取值为集合$C=\{c_1,...,c_k\}$，则可选取$C$的任意一个子集$C_0$作为判断条件，$x_{ij} \in C_0$的数据被分到左子结点，其他的分到右子结点。

叶子结点上最常使用的输出函数是常数预测值，这一预测值是基于分配到该叶子上的所有训练样本计算得到的。除了简单的常数预测值，也可以用更复杂的线性函数或者多项式。使用这些复杂的预测函数，也不可避免地增加了模型的整体复杂度，在训练样本不充分的时候，可能会导致过拟合。此外，决策树每片叶子上的预测结果仅仅由划分到该叶子上的训练数据拟合得到。决策树的叶子数目越多，决策树的拟合能力也越强，但是每片叶子上分配到的训练数据变得有限，这时叶子上的拟合结果会出现较大偏差。相反如果叶子数目太少，则拟合效果不佳，造成欠拟合。因此合理控制决策树的规模十分重要。

通过图1，也可以看到，决策树模型很轻易被人类理解，具有很强的可解释性。这也是决策树最大的优势。此外，在决策树中，没有复杂的数值计算，每一个特征在决策树中起到的作用清晰明了。我们很容易可以统计出每个特征被使用的次数、对数据拟合的贡献程度，从而对特征重要性作出评估。根据一个样本从根结点到叶子结点的路径，也容易得到每个样本的哪些特征对预测结果起到了什么样的贡献。这些对解释预测结果、分析特征与标签及特征之间的关系非常有帮助。


## 3. 决策树生长

在理解决策树的模型结构后，我们现在介绍决策树的生长算法。决策树的学习（生长）是一个贪心的、自顶向下的过程。和其他机器学习算法一样，决策树的学习也是数据驱动的。区别在于，它是一个不断切分数据的过程。决策树的算法流程如下：

    1.在训练的开始，决策树仅有一个根结点，所有的训练数据都被分配在根结点上（这时候的根结点也是叶子结点）。

    2.在当前所有叶子上，按照切分准则（Split Criterion）计算得到切分增益（Split Gain），枚举最优（切分增益最大）的切分条件。

    3.选择被切分的叶子。这里一般两种选择方法，一个是逐层生长（Level-Wise）和逐叶生长（Leaf-Wise）两种方式。逐层生长会切分当前所有叶子，逐叶生长则只会切分增益最大的那个叶子。

    4.根据第3步选中的叶子，生长决策树。如图2所示，把一个叶子结点变成中间结点，并长成两个新的叶子结点。并把原来结点上的数据，按照切分条件，分配到新的两个叶子上。

    5.循环2到4，知道满足终止条件。一般来说，决策树在达到预定义的复杂度，或者数据不能够再被切分时终止。

在这里面，核心的步骤是第二步的切分准则、枚举最优切分条件和第三步的切分叶子选择。我们会在先在接下来三个小节详细介绍它们。


---

<center> <img src=https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/439/22c89e2d9af64c88bdb7ab4915e2c357/WI-PTOSTNi90IBOAYnaNPQ.png> </center>

<center> 图2 决策树的切分示意图 </center>

---

### 3.1 切分准则

在决策树的发展历史中，有各种各样的切分准则被提了出来，如经典的ID3，C4.5等等。这些切分准则大多数都从信息论的角度设计得到的。然而，由于本书的视角主要从机器学习的角度，因此我们不会再详细介绍这些传统的切分方法。此外，这些方法在其他书籍都有很详细的介绍，有兴趣的同学可以自己查阅相关资料。现在我们基于损失函数，来推导决策树的切分准则。和其他机器学习算法类似，决策树的学习也需要不断地优化（降低）损失函数，让训练误差越来越小。因此，我们需要选择可以尽可能让损失函数降低最大的切分条件。基于这个原则，我们有如下的切分准则

$${\arg\max\limits_{l,\mathit{~~r}}}\left( {{\sum\limits_{x_{i}~ \in \text{leaf}~p}{L\left( {{\hat{y}}_{i},~y_{i}} \right)}} - ~{\sum\limits_{x_{i}~ \in \text{leaf}~l}{L\left( {{\hat{y}}_{i},~y_{i}} \right)}} - ~{\sum\limits_{x_{i}~ \in \text{leaf}~r}{L\left( {{\hat{y}}_{i},~y_{i}} \right)}}} \right)$$

其中,$L(\cdot, \cdot)$表示在单个样本上的损失函数，$\hat{y}_i$ 表示决策树模型对第$i$个样本的预测值。上面的式子表示，我们需要找到让叶子$p$切分成叶子$l$和$r$的损失函数的变化最大的切分条件。在有了这个一般化的切分准则后，现在我们把它细化到具体的损失函数。我们用回归问题中常见的均方差函数说起。
首先，我们先确定每个叶子的输出：假设叶子使用常数预测值，我们想让每个叶子上的均方差最小，即

$$a_{p}^{*} = {\arg{\min\limits_{a_{p}}{\sum\limits_{x_{i}~ \in \text{leaf}~p}\left( {y_{i} - a_{p}} \right)^{2}}}}$$

对于这个二次方程，我们很容易可以求解得到，当 $a_{p}^{*} = \frac{\sum_{x_{i}~ \in \text{leaf}~p}y_{i}}{\sum_{x_{i}~ \in \text{leaf}~p}1}$ 时,即分配到该叶子的样本的训练标签的均值，每个叶子上的均方差误差最小。
确定叶子的输出后，我们有切分准则为

$${\arg\max\limits_{l,\mathit{~~r}}}\left( {{\sum\limits_{x_{i}~ \in \text{leaf}~p}\left( {y_{i} - a_{p}^{*}} \right)^{2}} - ~{\sum\limits_{x_{i}~ \in \text{leaf}~l}\left( {y_{i} - a_{l}^{*}} \right)^{2}} - ~{\sum\limits_{x_{i}~ \in \text{leaf}~r}\left( {y_{i} - a_{r}^{*}} \right)^{2}}} \right)$$

经过一系列的展开和化简，且令$N_{p} = {\sum_{x_{i}~ \in \text{leaf}~p}1}$，$S_{p} = {\sum_{x_{i}~ \in \text{leaf}~p}y_{i}}$，分别表示每个叶子结点上的样本数量，和样本标签之和，我们可以得到简化后的切分准则为

$${\arg\max\limits_{l,\mathit{~~r}}}\left( {\frac{S_{l}^{2}}{N_{l}} + \frac{S_{r}^{2}}{N_{r}} - \frac{S_{p}^{2}}{N_{p}}} \right)$$

可以看到，这个化简后的形式非常简单。对于其他的损失函数，我们也可以用类似的方法处理，得到一个具体的切分准则。


### 3.2 切分条件枚举


在有了切分准则之后，我们可以利用它去枚举最优的切分条件。对于一个叶子来说，我们需要在分配到这叶子上的所有样本上的所有特征上的所有可能的切分条件里，找到能让切分准则最大的切分条件。如算法1所示，对于数值特征，切分条件是一个阈值，因此只需要枚举当前特征所有可能的取值即可（为了简洁，这里我们仅展示在根节点上对特征j的枚举过程。其他非根节点的枚举过程类似，但只需考虑该在叶子节点上的数据）；对于类别特征，切分条件是类别的子集。这时候的复杂度太高，为指数级别，在实践中很难使用（后面有时间的话，会详细介绍决策树如何高效地处理类别特征）。
 
对于一个特征来说，枚举所有切分条件的时间复杂度是$O(n^2)$，$n$为当前叶子的样本数量。这是因为，每次枚举一个切分条件，都需要过一次数据来得到累积的统计量（如算法1第8行到第12行所示），然后根据统计量计算切分增益，从而导致效率很低。


<center> <img src=https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/439/89c77797226448359ac57ad6891d41dd/-ui30ZeUcuJfFI5SiiEObA.png width=60%>

### 3.3 切分叶子选择

切分叶子的选择决定了决策树的生长方式。一般来说，常见的生长策略有两种：逐层生长（Level-Wise）和逐叶生长（Leaf-Wise）。

逐层生长会对当前的所有叶子进行切分。在这个过程中，决策树会逐层增长，如图3所示，因此被称为逐层生长的决策树。逐层生长的决策树总是一个完全二叉树。逐层生长的一个优点是，不会长出太深的树，模型的复杂度较低，不太容易导致过拟合。但逐层生长会让所有叶子生长的步调一致，导致效率较低，尤其当数据在叶子上的分布不平衡的时候。这时候，数据量较多的叶子本应当更充分地进行分割，但是却被迫与其他叶子保持一样的训练步伐，收敛速度慢，甚至可能产生欠拟合。此外，那些数据量小的叶子会被过度地分割，很容易在这部分数据上过拟合。



---
<center> <img src=https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/439/89c77797226448359ac57ad6891d41dd/dNL-CTZSX_o8XosxmO3h2Q.png> </center>

<center> 图3 逐层生长的生长策略 </center>

---

逐叶生长是一种更加贪心的方法，每一次只选取能让损失函数变化最大（切分增益最大）的那个叶子，如图4所示。相比于逐层生长的策略，逐叶生长产生的决策树结构更加灵活。在总叶子数相同的情况下，由于更激进的贪心策略，用逐叶生长方式学到的决策树上的训练误差也比逐层生长要小。但也因此，逐叶生长也更容易过拟合，需要更细致的调参。此外，我们也可以给逐叶生长的方法，加上深度的限制，避免长出太深的决策树从而导致过拟合。

---

<center> <img src=https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/439/89c77797226448359ac57ad6891d41dd/Or_W6WM8ESSNkFy3KgIdeg.png> </center>

<center> 图4 逐叶生长的生长策略 </center>

---

需要注意的是，在一些传统的决策树算法里，树是尽可能地长满，然后再用后剪枝的方法去掉一些没用的分支。在这些场景下，由于树都长满了，决策树的生长过程并不会影响最后的结果。但是，由于效率问题，现在的大多数决策树算法都不再使用这个方法。

## 4. 小结

可以看到，决策树的算法原理非常简单，仅需要一些简单的数学就能理解。但本文介绍也是最简单的决策树算法，由于其$O(n^2)$的复杂度，实践中效率很低，很难被真的用起来。我们会在后续的文章里，介绍针对决策树设计的高效率算法，预排序算法和直方图算法。敬请期待。